# Evaluation Average Execution Speed  
This Notebook calculates how fast (exec/sec) AFL runs certain applications

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def convert_unixtime_deltaseconds(df, unix=True, timecolumn='# unix_time'):
    if unix:
        df['date'] = pd.to_datetime(df[timecolumn],unit='s')
    else:
        df['date'] = pd.to_datetime(df[timecolumn])
        
    df['date'] = df['date'].dt.round('1s')
    return df

#https://stackoverflow.com/questions/25234941/python-regularise-irregular-time-series-with-linear-interpolation
def calc_avg_execs(df, event):
    df = convert_unixtime_deltaseconds(df)

    df = df[['date', ' execs_per_sec']]
    start_date = df.iloc[0]['date']
    end_date = start_date + pd.Timedelta(hours=24)

    df = df.append({'date': df['date'].iloc[0] + pd.Timedelta(hours=24)}, ignore_index=True)
    df[' execs_per_sec'] = df[' execs_per_sec'].ffill()

    df = df.set_index('date').resample('s').interpolate().resample('1h').asfreq().bfill()
    df = df.reset_index()
    return df

In [ ]:
resultspath = '../results/'
results = {
    'objdump': pd.DataFrame(),
    'nm': pd.DataFrame(),
    'ffmpeg': pd.DataFrame(),
    'mupdf': pd.DataFrame(),
    'harfbuzz': pd.DataFrame(),
    'gif2png': pd.DataFrame(),
    'size': pd.DataFrame(),
    'readelf': pd.DataFrame()
}
empty_seed = False

if empty_seed:
    seed=''
else:
    seed='pre'
    
for exec_name in results:
 
     for trial in range(1,6):
         #afl
         resultpath = os.path.join(resultspath + exec_name + seed, exec_name + seed + 'afl' + str(trial))
         df = pd.read_csv(resultpath)
         df = calc_avg_execs(df, 'AFL')
         results[exec_name] = results[exec_name].append(df)

In [71]:
for key in results:
    df = results[key]
    print(key + "\t" + str(round(df[' execs_per_sec'].mean(),1)))

objdump	997.0
nm	2148.1
ffmpeg	371.3
mupdf	889.5
harfbuzz	1965.4
gif2png	43.3
size	2278.1
readelf	2171.6
